# Compare FilterPy Kalman Filter to Matlab implementations

In [57]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import control as ct
import filterpy as fp
from filterpy.kalman import KalmanFilter

[print(f"{p.__name__}: {p.__version__}") for p in [np, pd, ct, fp]];

numpy: 1.25.2
pandas: 2.0.3
control: 0.9.4
filterpy: 1.4.5


In [42]:
#test_data_dir = "../process-observers/results"
test_data_dir = "test_data"
[name for name in os.listdir(test_data_dir) if name.endswith('.csv')]

['KF_sim_benchmark.csv']

In [44]:
filename = 'KF_sim_benchmark.csv'
test_data = pd.read_csv(os.path.join(test_data_dir, filename), index_col=0)
test_data.head()

,U,V,W_1,W_2,xNprocess_1,xNprocess_2,xNkalman_1,xNkalman_2
t,,,,,,,,
0,0,0.340051,0.460293,-0.397140,5.321222,5.321222,0.000000,0.000000
3,0,1.159851,0.054823,-0.243521,5.531515,4.924082,1.808274,1.768449
6,0,-1.428620,0.166245,-0.268474,5.694885,5.287994,3.649554,3.668464
9,0,0.545286,0.268365,0.330653,5.840366,5.426411,2.900010,3.095421
12,0,0.201605,0.937640,-0.086815,6.085418,6.171019,3.839475,4.067944


## Define System

MATLAB code from process-observers/sys_test_siso.m

In [58]:
Gc = ct.tf(2, np.convolve([10, 1], [15, 1]))
Gc

TransferFunction(array([2]), array([150,  25,   1]))

In [59]:
Ts = 3  # sample time
Gdss = ct.ss(ct.c2d(Gc, Ts, 'zoh'))
Gdss

<LinearIOSystem:sys[2]$sampled$converted:['u[0]']->['y[0]']>

In [78]:
# Check system matrices - note different state-space representation
# than Matlab
A = Gdss.A
B = Gdss.B
C = Gdss.C
D = Gdss.D

assert np.array_equal(A.round(8), [
    [ 1.55954897,  0.60653066],
    [-1.        ,  0.        ]
])
assert np.array_equal(B.round(8), [
    [1.],
    [0.]
])
assert np.array_equal(C.round(8), [
    [ 0.05088836, -0.04307501]
])
assert D == 0.0

# Dimensions
n = A.shape[0]  # number of states
nu = B.shape[1]  # number of inputs
ny = C.shape[0]  # number of outputs

# Covariance of process noise
Qp = np.diag([0.3, 0.2])

# Variance of measurement noise
Rp = 0.4

In [79]:
# Initial state of system
x0 = np.array([0.1, 0.5])

## Simulate system

In [39]:
n, nu, ny = 2, 1, 1
kf = KalmanFilter(dim_x=n, dim_z=ny)
kf.A = A
kf.A = B
kf.C = C
kf.D = D
kf.Q = Qp
kf.R = Rp
kf.x = x0
kf

KalmanFilter object
dim_x = 2
dim_z = 1
dim_u = 0
x = [0.1 0.5]
P = [[1. 0.]
     [0. 1.]]
x_prior = [[0. 0.]].T
P_prior = [[1. 0.]
           [0. 1.]]
x_post = [[0. 0.]].T
P_post = [[1. 0.]
          [0. 1.]]
F = [[1. 0.]
     [0. 1.]]
Q = [[0.3 0. ]
     [0.  0.2]]
R = 0.4
H = [[0. 0.]]
K = [[0. 0.]].T
y = [[0.]]
S = [[0.]]
SI = [[0.]]
M = [[0.]]
B = None
z = [[None]]
log-likelihood = -708.3964185322641
likelihood = 2.2250738585072014e-308
mahalanobis = 0.0
alpha = 1.0
inv = <function inv at 0x10fcd8fe0>

In [40]:
help(kf.update)

Help on method update in module filterpy.kalman.kalman_filter:

update(z, R=None, H=None) method of filterpy.kalman.kalman_filter.KalmanFilter instance
    Add a new measurement (z) to the Kalman filter.
    
    If z is None, nothing is computed. However, x_post and P_post are
    updated with the prior (x_prior, P_prior), and self.z is set to None.
    
    Parameters
    ----------
    z : (dim_z, 1): array_like
        measurement for this update. z can be a scalar if dim_z is 1,
        otherwise it must be convertible to a column vector.
    
    R : np.array, scalar, or None
        Optionally provide R to override the measurement noise for this
        one call, otherwise  self.R will be used.
    
    H : np.array, or None
        Optionally provide H to override the measurement function for this
        one call, otherwise self.H will be used.



In [ ]:
# number of points to simulate
nT = 100